In [0]:
# -*- coding: utf-8 -*-
"""
Created on Fri May 10 11:58:40 2019

@author: felip
"""
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from pandas import Series, DataFrame
from sklearn.model_selection import train_test_split
from sklearn import metrics 
from sklearn.metrics import classification_report
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import confusion_matrix

# KNN
model1 = KNeighborsClassifier(n_neighbors = 10)

# Arvore de Decisao
model2 = DecisionTreeClassifier()

# NB Gaussian
model3 = GaussianNB()


# Dataset
url = 'https://raw.githubusercontent.com/BigDataGal/Python-for-Data-Science/master/titanic-train.csv'
titanic = pd.read_csv(url)
titanic.columns = ['PassengerId','Survived','Pclass','Name','Sex','Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked']
titanic_data = titanic.drop(['PassengerId','Name','Ticket','Cabin'], 1)

# Funcao de agregacao
def age_approx(cols):
    Age = cols[0]
    Pclass = cols[1]
    
    if pd.isnull(Age):
        if Pclass == 1:
            return 37
        elif Pclass == 2:
            return 29
        else:
            return 24
    else:
        return Age

In [0]:
# Data preparation
titanic_data['Age'] = titanic_data[['Age', 'Pclass']].apply(age_approx, axis=1)
titanic_data.dropna(inplace=True)
gender = pd.get_dummies(titanic_data['Sex'],drop_first=True)
embark_location = pd.get_dummies(titanic_data['Embarked'],drop_first=True)
titanic_data.drop(['Sex', 'Embarked'],axis=1,inplace=True)
titanic_dmy = pd.concat([titanic_data,gender,embark_location],axis=1)
titanic_dmy.drop(['Pclass'],axis=1,inplace=True)
titanic_dmy.drop(['Q'],axis=1,inplace=True)

X = titanic_dmy.iloc[:,[1,2,3,4,5,6]].values
y = titanic_dmy.iloc[:,0].values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state=25)

In [4]:
#Testanto o knn apenas
knn = KNeighborsClassifier(n_neighbors = 10)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
print(classification_report(y_test, y_pred))
print(metrics.accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.84      0.79       164
           1       0.68      0.54      0.61       103

    accuracy                           0.73       267
   macro avg       0.71      0.69      0.70       267
weighted avg       0.72      0.73      0.72       267

0.7265917602996255


In [5]:
#Testanto Arvore de decisão  apenas
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
print(classification_report(y_test, y_pred))
print(metrics.accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.79      0.81       164
           1       0.68      0.74      0.71       103

    accuracy                           0.77       267
   macro avg       0.76      0.76      0.76       267
weighted avg       0.77      0.77      0.77       267

0.7677902621722846


In [6]:
#Testanto Arvore de Naive Bayes
nb = GaussianNB()
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
print(classification_report(y_test, y_pred))
print(metrics.accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.83      0.82       164
           1       0.72      0.69      0.70       103

    accuracy                           0.78       267
   macro avg       0.76      0.76      0.76       267
weighted avg       0.77      0.78      0.77       267

0.7752808988764045


In [7]:
#Esemble baseado em voto marjoritário
model = VotingClassifier(estimators=[('knn', model1), ('dt', model2), ('gnb', model3)], voting='hard')
model.fit(X_train,y_train)
model.score(X_test,y_test)


#Ensemble baseado em pesos e na probabilidade da classes
knn.fit(X_train,y_train)
nb.fit(X_train,y_train)
dt.fit(X_train,y_train)

pred1=knn.predict_proba(X_test)
pred2=nb.predict_proba(X_test)
pred3=dt.predict_proba(X_test)

finalpred=(pred1*0.2+pred2*0.5+pred3*0.3)

for i in range(0, len(finalpred[:,1])):
    if finalpred[i,0] > finalpred[i,1]:
        y_pred[i] = 0
    else:
        y_pred[i] = 1
        
print(metrics.accuracy_score(y_test, y_pred))

0.7790262172284644


In [9]:
#Ramdon Forest
from sklearn.ensemble import RandomForestClassifier
modelRfc= RandomForestClassifier(random_state=1)
modelRfc.fit(X_train, y_train)
modelRfc.score(X_test,y_test)

#Exibe as caracteristicas mais relevantes para o ramdom forest
for i, j in sorted(zip(titanic_dmy.iloc[:,[1,2,3,4,5,6]].columns, modelRfc.feature_importances_)):
    print(i, j)

Age 0.2732323925379839
Fare 0.3231084121670654
Parch 0.04615874472138822
S 0.037427619958221324
SibSp 0.05552116620691575
male 0.2645516644084253


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [10]:
#Boosting utilizando  o algoritmo AdaBoost
from sklearn.ensemble import AdaBoostClassifier
modelADB = AdaBoostClassifier(random_state=1, n_estimators = 25)
modelADB.fit(X_train, y_train)
modelADB.score(X_test,y_test)

0.7715355805243446